In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from causal_models.copd_data_scms import CopdDataSCM
from gmm.copd_data import LinearCombinedDataCrossFitNuisanceEstimator
from gmm.gmm_equations import GMMEstimator
from gmm.observational_two_covariates import ObservationalTwoCovariatesGMMEqs

In [4]:
REPO_DIR = "/path/to/repo/directory"
DATA_DIR = os.path.join(REPO_DIR, "datasets", "yang_and_ding")
DATA_VAL_FILEPATH = os.path.join(DATA_DIR, "validation_ns3.csv")
DATA_MAIN_FILEPATH = os.path.join(DATA_DIR, "main_ns3.csv")

In [6]:
def get_combined_ate():
    true_scm_val = CopdDataSCM(data_filepath=DATA_VAL_FILEPATH)
    true_scm_main = CopdDataSCM(data_filepath=DATA_MAIN_FILEPATH)

    df_val = true_scm_val.get_original_dataset()
    df_val["SEL"] = 1

    df_main = true_scm_main.get_original_dataset()
    df_main["SEL"] = 0

    df = (
        pd.concat([df_val, df_main])
        .sample(frac=1, replace=False)
        .reset_index(drop=True)
    )

    nuisance = LinearCombinedDataCrossFitNuisanceEstimator()
    gmm = GMMEstimator(
        df=df, gmm_eqs=ObservationalTwoCovariatesGMMEqs(), nuisance=nuisance
    )

    params, moment_covariance = gmm.find_parameters(num_iters=2)
    optimal_kappa = gmm.find_optimal_k(
        moment_covariance=moment_covariance, params=params, cost_per_source=[4, 1]
    )
    return params[0], optimal_kappa

In [7]:
random_seed = 232281293
np.random.seed(random_seed)
results = [get_combined_ate() for i in tqdm(range(2000))]
ate_vals = [r[0] for r in results]
optimal_kappas = [r[1] for r in results]
np.random.seed(None)

100%|██████████| 2000/2000 [05:31<00:00,  6.04it/s]


In [8]:
ate_vals = np.array(ate_vals)
mean = np.mean(ate_vals)
ci = 1.96 * np.sqrt(np.var(ate_vals) / len(ate_vals))
print(f"ATE = {mean} +- {ci}")

ATE = 0.015609412706598623 +- 3.518419461558675e-05


In [9]:
optimal_kappas = np.array(optimal_kappas)
mean = np.mean(optimal_kappas)
ci = 1.96 * np.sqrt(np.var(optimal_kappas) / len(optimal_kappas))
print(f"optimal_kappas = {mean} +- {ci}")

optimal_kappas = 0.10764016102577924 +- 0.001578670833787042
